# Exercise 4.7: Deriving New Variables

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import os

## Load merged orders–products dataset

In [4]:
from pathlib import Path

# Define prepared data path
PREPARED_DATA = Path(
    r"F:\Data Analytics\Projects\Instacart_Basket_Analysis\Instacart Basket Analysis 01-05-2026\02 Data\Prepared Data"
)

In [5]:
# Load merged orders-products dataset
ords_prods_merge = pd.read_pickle(
    PREPARED_DATA / "ords_prods_merge.pkl"
)

In [6]:
ords_prods_merge.shape

(32641268, 14)

In [7]:
# Create a working subset (first 1 million rows)
df = ords_prods_merge[:1000000]

In [8]:
df.shape

(1000000, 14)

# Define price range labeling function

In [11]:
# Define function to label products by price range
def price_label(row):
    if row['prices'] <= 5:
        return 'Low-range product'
    elif (row['prices'] > 5) and (row['prices'] <= 15):
        return 'Mid-range product'
    elif row['prices'] > 15:
        return 'High-range product'
    else:
        return np.nan


In [12]:
# Apply price range labels to the working subset
df['price_range'] = df.apply(price_label, axis=1)

C:\Users\sheid\AppData\Local\Temp\ipykernel_30108\3874836084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [13]:
df['price_range'].value_counts(dropna=False)

price_range
Mid-range product     668114
Low-range product     312168
High-range product     12340
NaN                     7378
Name: count, dtype: int64

In [14]:
df['price_range_loc'] = np.nan

C:\Users\sheid\AppData\Local\Temp\ipykernel_30108\1455241719.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range_loc'] = np.nan


In [15]:
# Create a working subset (first 1 million rows)
df = ords_prods_merge[:1000000]

# Explicitly create a copy to avoid SettingWithCopyWarning
df = df.copy()

In [16]:
df['price_range_loc'] = np.nan

In [17]:
# Label high-range products (price > 15)
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

C:\Users\sheid\AppData\Local\Temp\ipykernel_30108\2853355232.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'High-range product' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'


In [18]:
df['price_range_loc'] = df['price_range_loc'].astype('object')

In [19]:
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [20]:
# Label low-range products (price ≤ 5)
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

# Label mid-range products (5 < price ≤ 15)
df.loc[(df['prices'] > 5) & (df['prices'] <= 15), 'price_range_loc'] = 'Mid-range product'

In [21]:
df['price_range_loc'].value_counts(dropna=False)

price_range_loc
Mid-range product     668114
Low-range product     312168
High-range product     12340
NaN                     7378
Name: count, dtype: int64

### Price Range Categorization

A new variable, `price_range_loc`, was created to categorize products into price tiers based on their listed price:

- **Low-range product:** price ≤ $5  
- **Mid-range product:** $5 < price ≤ $15  
- **High-range product:** price > $15  

The majority of products fall within the mid-range category, with fewer products in the low- and high-range categories. A small number of rows remain uncategorized due to missing price values, which were intentionally left as NaN.

In [22]:
df['prices'].max()

99999.0

### Price Validation Check

A maximum price value of 99,999 was identified in the dataset. This value represents a placeholder for missing or unavailable pricing rather than a true product price. These rows were retained in the dataset but excluded from meaningful price categorization, resulting in some values remaining as NaN in the price range variable.

In [23]:
(df['prices'] == 99999).sum()

np.int64(15)

In [24]:
df[['prices', 'price_range_loc']].head()

,prices,price_range_loc
0,9.0,Mid-range product
1,12.5,Mid-range product
2,4.4,Low-range product
3,4.7,Low-range product
4,1.0,Low-range product


In [25]:
df.loc[df['price_range_loc'] == 'High-range product', 
       ['prices', 'price_range_loc']].head()

,prices,price_range_loc
403,19.6,High-range product
406,19.6,High-range product
442,23.2,High-range product
471,23.2,High-range product
486,23.2,High-range product


## Price Range Validation and Logic Review

Using `.loc` is the preferred approach for deriving new variables in large datasets because it avoids row-wise Python loops and operates directly on vectorized pandas operations. This improves performance, reduces memory overhead, and prevents unintended behavior such as SettingWithCopyWarning that can occur when working on dataframe slices.

In [27]:
df['price_range_loc'].value_counts(dropna=False)

price_range_loc
Mid-range product     668114
Low-range product     312168
High-range product     12340
NaN                     7378
Name: count, dtype: int64

### Deriving Order Day Activity Labels (For-Loops)

In this section, a for-loop is used to derive a new categorical variable that summarizes how busy each day of the week is based on order frequency.

In [29]:
# Check frequency of orders by day of week
df['orders_day_of_week'].value_counts()

orders_day_of_week
0    190055
1    169641
6    139701
5    133960
2    130391
3    118358
4    117894
Name: count, dtype: int64

In [30]:
# Create empty list to store day-busy labels
result = []

In [31]:
# Loop through each value in orders_day_of_week
for value in df['orders_day_of_week']:
    
    if value == 0:
        result.append('Busiest day')
        
    elif value == 4:
        result.append('Least busy')
        
    else:
        result.append('Regularly busy')

In [32]:
# Add day-busy labels to the dataframe
df['day_busy'] = result

In [33]:
df['day_busy'].value_counts()

day_busy
Regularly busy    692051
Busiest day       190055
Least busy        117894
Name: count, dtype: int64

## Exercise 4.7 – Task

### Task Step 1 – Confirm Required Columns

In [34]:
# Step 1 check: confirm required columns exist
df[['prices', 'price_range_loc', 'orders_day_of_week', 'day_busy']].head()

,prices,price_range_loc,orders_day_of_week,day_busy
0,9.0,Mid-range product,2,Regularly busy
1,12.5,Mid-range product,2,Regularly busy
2,4.4,Low-range product,2,Regularly busy
3,4.7,Low-range product,2,Regularly busy
4,1.0,Low-range product,2,Regularly busy


### Task Step 2 – Identify Two Busiest and Two Slowest Days

df['orders_day_of_week'].value_counts()

In [37]:
# Create new column for updated day-busy labels
df['day_busy_v2'] = 'Regularly busy'

# Label two busiest days
df.loc[df['orders_day_of_week'].isin([0, 1]), 'day_busy_v2'] = 'Busiest days'

# Label two slowest days
df.loc[df['orders_day_of_week'].isin([3, 4]), 'day_busy_v2'] = 'Slowest days'

In [38]:
df['day_busy_v2'].value_counts()

day_busy_v2
Regularly busy    404052
Busiest days      359696
Slowest days      236252
Name: count, dtype: int64

### Task Step 3 – Validation and Observations

The updated `day_busy_v2` column correctly assigns:
- “Busiest days” to the two most frequently occurring order days,
- “Slowest days” to the two least frequent order days, and
- “Regularly busy” to all remaining days.

A frequency check confirms that all rows are labeled and no missing values are present.  
This validation was performed on a 1,000,000-row working subset prior to applying the same logic to the full dataset.


In [40]:
# Task Step 4 – Explore order frequency by hour of day
df['order_hour_of_day'].value_counts().sort_index()

order_hour_of_day
0      5991
1      2949
2      1879
3      1499
4      1680
5      2698
6      9531
7     27845
8     53703
9     77860
10    86317
11    84710
12    80592
13    81600
14    81319
15    81171
16    76880
17    65362
18    51519
19    39497
20    29707
21    24551
22    19142
23    11998
Name: count, dtype: int64

In [41]:
# Task Step 4: Create busiest_period_of_day column
# Default all rows to 'Average orders'

df['busiest_period_of_day'] = 'Average orders'

In [42]:
# Label peak demand hours as 'Most orders'
# Based on highest order frequencies between 10 AM and 4 PM

df.loc[
    df['order_hour_of_day'].isin([10, 11, 12, 13, 14, 15, 16]),
    'busiest_period_of_day'
] = 'Most orders'

In [43]:
# Label lowest demand hours as 'Fewest orders'
# Late night and early morning periods

df.loc[
    df['order_hour_of_day'].isin([0, 1, 2, 3, 4, 5, 6, 20, 21, 22, 23]),
    'busiest_period_of_day'
] = 'Fewest orders'

In [44]:
# Validate busiest_period_of_day distribution

df['busiest_period_of_day'].value_counts()

busiest_period_of_day
Most orders       572589
Average orders    315786
Fewest orders     111625
Name: count, dtype: int64

## Task Step 5 – Frequency of Busiest Period of Day

In [45]:
# Task Step 5: Print frequency of busiest_period_of_day
df['busiest_period_of_day'].value_counts()

busiest_period_of_day
Most orders       572589
Average orders    315786
Fewest orders     111625
Name: count, dtype: int64

**Note:**  
This analysis was performed on a 1,000,000-row working subset of the full Instacart dataset to validate logic and performance. The same grouping rules and labeling logic are intended to be applied to the complete dataset prior to final reporting.

In [48]:
# Task Step 7: Export dataframe as pickle file to Prepared Data folder

import os

# Define correct export path (project-specific Prepared Data folder)
export_path = r"F:\Data Analytics\Projects\Instacart_Basket_Analysis\Instacart Basket Analysis 01-05-2026\02 Data\Prepared Data"
file_name = "instacart_prepared_exercise_4_7.pkl"

# Create folder if it does not exist
os.makedirs(export_path, exist_ok=True)

# Export dataframe
df.to_pickle(os.path.join(export_path, file_name))

# Confirm export
os.listdir(export_path)

['departments_wrangled.csv',
 'orders_checked.csv',
 'orders_products_combined.pkl',
 'orders_wrangled.csv',
 'ords_prods_merge.pkl',
 'products_checked.csv',
 'instacart_prepared_exercise_4_7.pkl']